In [2]:
# Dependencies
from flask import Flask

In [3]:
# Create a new flask app instance
app = Flask(__name__)

In [ ]:
# Create flask routes
@app.route('/')
def hello_world():
    return 'Hello world'